In [ ]:
import pandas as pd
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
    df = pd.read_csv(filename)

df.head()


Saving IMDB Dataset.csv to IMDB Dataset (1).csv


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
import re
import string

def clean_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)  # HTML etiketlerini sil
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Noktalama
    text = re.sub(r"\d+", "", text)  # Sayılar
    text = re.sub(r"\s+", " ", text).strip()  # Boşluk düzeltmesi
    return text

df['clean_review'] = df['review'].apply(clean_text)
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Eğitim ve test ayrımı
X_train, X_test, y_train, y_test = train_test_split(df['clean_review'], df['label'], test_size=0.2, random_state=42)

# TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Model
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# Başarım
y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4961
           1       0.89      0.91      0.90      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [ ]:


import gradio as gr

def predict_sentiment(text):
    cleaned = clean_text(text)
    vec = vectorizer.transform([cleaned])
    pred = model.predict(vec)[0]
    return "Positive " if pred == 1 else "Negative "

# Arayüz tanımı
iface = gr.Interface(fn=predict_sentiment,
                     inputs=gr.Textbox(lines=3, placeholder="Bir yorum yazın..."),
                     outputs="text",
                     title="Duygu Analizi",
                     description="Bir cümle yazın, model tahmin etsin!")

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a41f85aad8ee5d5094.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import random

# Rastgele 5 örnek seç
sample = df.sample(5)

for i, row in sample.iterrows():
    original_text = row['review']
    cleaned_text = clean_text(original_text)
    vectorized = vectorizer.transform([cleaned_text])
    prediction = model.predict(vectorized)[0]

    predicted_label = "Positive 😊" if prediction == 1 else "Negative 😠"
    actual_label = "Positive" if row['label'] == 1 else "Negative"

    print("🔹 Cümle:", original_text[:200], "...")
    print("✅ Gerçek:", actual_label)
    print("🤖 Tahmin:", predicted_label)
    print("-" * 80)


🔹 Cümle: Rivalry between brothers leads to main story line. Navy Commander Chuck Prescott(Marshall Thompson)has developed the Y12 aircraft to test how far man can go up in the atmosphere. His brother, Lt. Dan  ...
✅ Gerçek: Negative
🤖 Tahmin: Negative 😠
--------------------------------------------------------------------------------
🔹 Cümle: I liked the movie, first of all because it told an interesting story, but the story as told in the movie felt like it was condensed from a much-longer story. Since the book is over 400 pages, that mak ...
✅ Gerçek: Positive
🤖 Tahmin: Positive 😊
--------------------------------------------------------------------------------
🔹 Cümle: The film begins with a dowdy housewife (Norma Shearer) finding out that her husband (Rod La Rocque) had been cheating on her. Three years pass and apparently they'd been divorced during these years du ...
✅ Gerçek: Positive
🤖 Tahmin: Positive 😊
----------------------------------------------------------------------------